In [1]:
# %pip install altair_saver seaborn altair

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(style='white')
import numpy as np
import pandas as pd
import altair as alt
import glob
import json
from altair_saver import save as alt_save

pd.set_option('display.max_columns', None)

## thesis

1. вариантов того, как проходит 1ый год много
2. разброс карьерных траекторий для 8го номера драфта - большой
3. первые три сезона - рост показателей

## questions 
1. как распределены по весу\росту\позиции\возрасту
2. сколько минут получают от сезона к сезону
3. сколько очков набирают по сезонам
4. как меняется кол-во минут\очков\подборов\передач по неделям для первого сезона

# load data

In [3]:
df_cpi = pd.read_csv('data/commonplayerinfo_0.csv')
df_dc = pd.read_csv('data/draftcombineplayeranthro.csv')
df_cpi = pd.merge(df_cpi, df_dc[['PLAYER_ID', 'HEIGHT_WO_SHOES', 'WEIGHT']], how='left', left_on='player_id', right_on='PLAYER_ID' )
df_cpi

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,HEIGHT,WEIGHT_x,SEASON_EXP,JERSEY,POSITION,ROSTERSTATUS,GAMES_PLAYED_CURRENT_SEASON_FLAG,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_CITY,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG,player_id,PLAYER_ID,HEIGHT_WO_SHOES,WEIGHT_y
0,2037,Jamal,Crawford,Jamal Crawford,"Crawford, Jamal",J. Crawford,jamal-crawford,1980-03-20T00:00:00,Michigan,USA,Michigan/USA,6-5,200,21,1.0,Guard,Inactive,N,1610612746,Clippers,LAC,clippers,LA,jamal_crawford,2000,2019,N,Y,Y,2000,1,8,N,2037,2037.0,76.50,175.0
1,2205,DeSagana,Diop,DeSagana Diop,"Diop, DeSagana",D. Diop,desagana-diop,1982-01-30T00:00:00,Oak Hill Academy (VA),Senegal,Oak Hill Academy (VA)/Senegal,7-0,280,13,2.0,Center,Inactive,N,1610612742,Mavericks,DAL,mavericks,Dallas,desagana_diop,2001,2012,N,Y,Y,2001,1,8,N,2205,2205.0,82.00,314.0
2,2404,Chris,Wilcox,Chris Wilcox,"Wilcox, Chris",C. Wilcox,chris-wilcox,1982-09-03T00:00:00,Maryland,USA,Maryland/USA,6-10,235,12,44.0,Forward-Center,Inactive,N,1610612746,Clippers,LAC,clippers,Los Angeles,chris_wilcox,2002,2012,N,Y,Y,2002,1,8,N,2404,2404.0,80.25,218.0
3,2551,T.J.,Ford,T.J. Ford,"Ford, T.J.",T. Ford,tj-ford,1983-03-24T00:00:00,Texas,USA,Texas/USA,6-0,165,8,NaN,Guard,Inactive,N,0,NaN,NaN,NaN,NaN,t.j._ford,2003,2011,N,Y,Y,2003,1,8,N,2551,2551.0,71.00,162.0
4,2737,Rafael,Araujo,Rafael Araujo,"Araujo, Rafael",R. Araujo,rafael-araujo,1980-08-12T00:00:00,Brigham Young,Brazil,Brigham Young/Brazil,6-11,270,4,55.0,Center,Inactive,N,1610612761,Raptors,TOR,raptors,Toronto,rafael_araujo,2004,2006,N,Y,Y,2004,1,8,N,2737,NaN,NaN,NaN
5,101112,Channing,Frye,Channing Frye,"Frye, Channing",C. Frye,channing-frye,1983-05-17T00:00:00,Arizona,USA,Arizona/USA,7-0,255,14,9.0,Forward-Center,Inactive,N,1610612756,Suns,PHX,suns,Phoenix,channing_frye,2005,2018,N,Y,Y,2005,1,8,N,101112,101112.0,81.50,243.6
6,200752,Rudy,Gay,Rudy Gay,"Gay, Rudy",R. Gay,rudy-gay,1986-08-17T00:00:00,Connecticut,USA,Connecticut/USA,6-8,250,18,22.0,Forward-Guard,Inactive,N,1610612762,Jazz,UTA,jazz,Utah,rudy_gay,2006,2022,N,Y,Y,2006,1,8,N,200752,200752.0,79.00,222.0
7,201148,Brandan,Wright,Brandan Wright,"Wright, Brandan",B. Wright,brandan-wright,1987-10-05T00:00:00,North Carolina,USA,North Carolina/USA,6-10,235,10,NaN,Forward-Center,Inactive,N,0,NaN,NaN,NaN,NaN,brandan_wright,2007,2017,N,Y,Y,2007,1,8,N,201148,201148.0,80.75,200.0
8,201570,Joe,Alexander,Joe Alexander,"Alexander, Joe",J. Alexander,joe-alexander,1986-12-26T00:00:00,West Virginia,USA,West Virginia/USA,6-8,230,3,20.0,Forward,Inactive,N,1610612749,Bucks,MIL,bucks,Milwaukee,joe_alexander,2008,2009,Y,Y,Y,2008,1,8,N,201570,201570.0,79.25,220.0
9,201941,Jordan,Hill,Jordan Hill,"Hill, Jordan",J. Hill,jordan-hill,1987-07-27T00:00:00,Arizona,USA,Arizona/USA,6-10,240,9,27.0,Forward-Center,Inactive,N,1610612747,Lakers,LAL,lakers,Los Angeles,jordan_hill,2009,2016,N,Y,Y,2009,1,8,N,201941,201941.0,81.25,232.4


In [4]:
import os
import json
import pandas as pd

# parse MIN "MM:SS" -> minutes float
def parse_min(s):
    try:
        if pd.isna(s):
            return np.nan
        if isinstance(s, (int, float)):
            return float(s)
        if ":" in str(s):
            m, sec = str(s).split(":")
            return float(m) + float(sec) / 60.0
        return float(s)
    except Exception:
        return np.nan

def load_all_boxscores(root="data/bs"):
    """
    Walk through data/bs/<player_id>/<season>/*.json and load boxscores.

    Supports two formats:
    - Old stats.nba.com format with 'resultSets' -> 'PlayerStats'
    - New nba.cloud format with 'boxScoreTraditional'

    Returns a single concatenated DataFrame with columns matching the old
    'PlayerStats' schema where possible.
    """
    records = []

    # Column order to normalize to (old format headers)
    target_cols = [
        "GAME_ID", "TEAM_ID", "TEAM_ABBREVIATION", "TEAM_CITY",
        "PLAYER_ID", "PLAYER_NAME", "NICKNAME", "START_POSITION",
        "COMMENT", "MIN", "FGM", "FGA", "FG_PCT", "FG3M", "FG3A",
        "FG3_PCT", "FTM", "FTA", "FT_PCT", "OREB", "DREB", "REB",
        "AST", "STL", "BLK", "TO", "PF", "PTS", "PLUS_MINUS"
    ]

    for player_id in os.listdir(root):
        p_path = os.path.join(root, player_id)
        if not os.path.isdir(p_path):
            continue

        for season in os.listdir(p_path):
            s_path = os.path.join(p_path, season)
            if not os.path.isdir(s_path):
                continue

            # All game JSON files
            for fname in os.listdir(s_path):
                if not fname.endswith(".json"):
                    continue

                fpath = os.path.join(s_path, fname)
                try:
                    with open(fpath, "rt") as f:
                        data = json.load(f)

                    if "resultSets" in data:  # Old format
                        rs = None
                        for r in data.get("resultSets", []):
                            if r.get("name") == "PlayerStats":
                                rs = r
                                break
                        if rs is None:
                            # No applicable result set
                            continue

                        df = pd.DataFrame(rs["rowSet"], columns=rs["headers"])
                        # Keep only the player directory's player
                        df = df[df['PLAYER_ID'].astype(str) == player_id].copy()
                        if df.empty:
                            continue

                    elif "boxScoreTraditional" in data:  # New format
                        bst = data["boxScoreTraditional"]
                        game_id = bst.get("gameId")

                        def team_rows(team_key):
                            team = bst.get(team_key, {})
                            team_id = team.get("teamId")
                            team_city = team.get("teamCity")
                            team_tri = team.get("teamTricode")
                            players = team.get("players", [])
                            rows = []
                            for p in players:
                                stats = p.get("statistics", {})
                                row = {
                                    "GAME_ID": game_id,
                                    "TEAM_ID": team_id,
                                    "TEAM_ABBREVIATION": team_tri,
                                    "TEAM_CITY": team_city,
                                    "PLAYER_ID": p.get("personId"),
                                    "PLAYER_NAME": f"{p.get('firstName','')} {p.get('familyName','')}".strip(),
                                    "NICKNAME": p.get("firstName"),
                                    "START_POSITION": p.get("position"),
                                    "COMMENT": p.get("comment", ""),
                                    "MIN": stats.get("minutes"),
                                    "FGM": stats.get("fieldGoalsMade"),
                                    "FGA": stats.get("fieldGoalsAttempted"),
                                    "FG_PCT": stats.get("fieldGoalsPercentage"),
                                    "FG3M": stats.get("threePointersMade"),
                                    "FG3A": stats.get("threePointersAttempted"),
                                    "FG3_PCT": stats.get("threePointersPercentage"),
                                    "FTM": stats.get("freeThrowsMade"),
                                    "FTA": stats.get("freeThrowsAttempted"),
                                    "FT_PCT": stats.get("freeThrowsPercentage"),
                                    "OREB": stats.get("reboundsOffensive"),
                                    "DREB": stats.get("reboundsDefensive"),
                                    "REB": stats.get("reboundsTotal"),
                                    "AST": stats.get("assists"),
                                    "STL": stats.get("steals"),
                                    "BLK": stats.get("blocks"),
                                    "TO": stats.get("turnovers"),
                                    "PF": stats.get("foulsPersonal"),
                                    "PTS": stats.get("points"),
                                    "PLUS_MINUS": stats.get("plusMinusPoints"),
                                }
                                rows.append(row)
                            return rows

                        rows = team_rows("homeTeam") + team_rows("awayTeam")
                        if not rows:
                            continue
                        df = pd.DataFrame(rows)

                        # Filter for the player directory's player
                        df = df[df['PLAYER_ID'].astype(str) == player_id].copy()
                        if df.empty:
                            continue

                        # Ensure all target columns exist
                        for col in target_cols:
                            if col not in df.columns:
                                df[col] = pd.NA

                        # Reorder columns to target order
                        df = df[target_cols]

                    else:
                        # Unknown format; skip
                        continue

                    # Attach file context
                    df["player_dir"] = player_id
                    df["season_dir"] = season
                    df["game_file"] = fname

                    df["MIN_float"] = df["MIN"].apply(parse_min)
                    records.append(df)

                except Exception as e:
                    print(f"Failed to load {fpath}: {e}")

    if not records:
        return pd.DataFrame(columns=target_cols + ["player_dir", "season_dir", "game_file"])

    return pd.concat(records, ignore_index=True)
df_bs = load_all_boxscores('data/bs')
print(df_bs.shape)
df_bs.head()

(1532, 33)


,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,player_dir,season_dir,game_file,MIN_float
0,0021300386,1610612765,DET,Detroit,203484,Kentavious Caldwell-Pope,Kentavious,G,,22:18,1.0,2.0,0.500,1.0,2.0,0.5,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0,2.0,5.0,-7.0,203484,2013-14,0021300386.json,22.300000
1,0021300532,1610612765,DET,Detroit,203484,Kentavious Caldwell-Pope,Kentavious,G,,30:10,5.0,11.0,0.455,2.0,4.0,0.5,0.0,0.0,0.0,2.0,5.0,7.0,2.0,3.0,0.0,1.0,3.0,12.0,-6.0,203484,2013-14,0021300532.json,30.166667
2,0021300220,1610612765,DET,Detroit,203484,Kentavious Caldwell-Pope,Kentavious,G,,18:51,0.0,4.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,3.0,0.0,1.0,1.0,0.0,-6.0,203484,2013-14,0021300220.json,18.850000
3,0021300498,1610612765,DET,Detroit,203484,Kentavious Caldwell-Pope,Kentavious,G,,26:41,6.0,8.0,0.750,2.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,14.0,-9.0,203484,2013-14,0021300498.json,26.683333
4,0021300272,1610612765,DET,Detroit,203484,Kentavious Caldwell-Pope,Kentavious,G,,27:54,3.0,6.0,0.500,2.0,4.0,0.5,0.0,2.0,0.0,2.0,1.0,3.0,1.0,0.0,1.0,1.0,1.0,8.0,7.0,203484,2013-14,0021300272.json,27.900000


In [5]:
(df_bs.PLAYER_ID.astype(str) == '1642856').sum()

35

## game number

In [6]:
import pandas as pd
import glob

def get_team_game_numbers(path_pattern='data/gamefinder/*.csv'):
    # Load all CSV files matching the pattern
    all_files = glob.glob(path_pattern)
    df_list = [pd.read_csv(f) for f in all_files]

    if not df_list:
        return {}

    df = pd.concat(df_list, ignore_index=True)

    # Ensure GAME_DATE is datetime for proper sorting
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

    # Sort by date to ensure the cumulative count represents the order of games
    df = df.sort_values('GAME_DATE')

    # Group by TEAM_ID and SEASON_ID, then calculate cumulative count
    # Note: cumcount() starts at 0, so we add 1 for "1st game", "2nd game", etc.
    df['GAME_NUMBER'] = df.groupby(['TEAM_ID', 'SEASON_ID']).cumcount() + 1
    df["WEEK"] = (df["GAME_NUMBER"] // 4).astype(int)
    df["GAMES_ON_WEEK"] = df.groupby(["SEASON_ID", "TEAM_ID", "WEEK"])["GAME_ID"].transform("nunique")

    # Create the dictionary mapping (TEAM_ID, GAME_ID) -> GAME_NUMBER
    # We use zip to efficiently build the dictionary
    game_num_dict = dict(zip(zip(df['TEAM_ID'], df['GAME_ID']),
                             zip(df['GAME_NUMBER'], df['WEEK'], df["GAMES_ON_WEEK"])))

    return game_num_dict

# Usage:
game_numbers = get_team_game_numbers()
list(game_numbers.items())[-5:]

[((1610612754, 22500595), (43, 10, 4)),
 ((1610612742, 22500593), (43, 10, 4)),
 ((1610612744, 22500599), (43, 10, 4)),
 ((1610612763, 22500602), (41, 10, 2)),
 ((1610612753, 22500602), (42, 10, 3))]

In [7]:
df_bs["team_game_number"] = [game_numbers[(idx[0], int(idx[1]))][0]
                             for idx in zip(df_bs["TEAM_ID"], df_bs["GAME_ID"])]
df_bs["team_week"] = [game_numbers[(idx[0], int(idx[1]))][1]
                             for idx in zip(df_bs["TEAM_ID"], df_bs["GAME_ID"])]
df_bs["team_games_on_week"] = [game_numbers[(idx[0], int(idx[1]))][2]
                             for idx in zip(df_bs["TEAM_ID"], df_bs["GAME_ID"])]
df_bs["team_games_on_week"].value_counts(dropna=False)

team_games_on_week
4    1441
3      90
1       1
Name: count, dtype: int64

In [8]:
import pandas as pd
import numpy as np

def build_player_week_df(df_all):
    # Work on a copy
    df = df_all.copy()

    # --- normalize columns (adapt if your column names differ) ---
    # Ensure GAME_ID is sortable numeric (if like '0022100045', int is fine)
    df["GAME_ID"] = df["GAME_ID"].astype(int)

    # sort so cumcount / ordering is stable
    df = df.sort_values(["player_dir", "season_dir", "PLAYER_ID", "GAME_ID"]).reset_index(drop=True)

    # player-local game index (0-based)
    df["player_game_number"] = df.groupby(["player_dir", "season_dir", "PLAYER_ID"]).cumcount()

    df["week_player"] = (df["player_game_number"] // 4).astype(int)

    # --- weekly (simple) aggregations per player-week ---
    # games_played, games_started, weekly means for MIN,PTS,REB,AST
    # assume START_POSITION empty string or NaN when not started
    df["started_flag"] = df["START_POSITION"].fillna("").astype(str).str.strip().replace("", "0")
    # make started_flag numeric 1/0: non-empty means started (adjust if your data uses 'G' or different)
    df["started_flag"] = np.where(df["START_POSITION"].notna() & (df["START_POSITION"].astype(str).str.strip() != ""), 1, 0)

    weekly_player = (
        df
        .groupby(["player_dir", "season_dir", "PLAYER_ID", "PLAYER_NAME", "TEAM_ID", "team_week"], as_index=False)
        .agg(
              games_played=("GAME_ID", "nunique"),
              games_started=("started_flag", "sum"),
              min_mean_week=("MIN_float", "mean"),
              pts_mean_week=("PTS", "mean"),
              reb_mean_week=("REB", "mean"),
              ast_mean_week=("AST", "mean"),
              team_games_this_week=("team_games_on_week", "first"),
              pts_sum_week=("PTS", "sum"),
              min_sum_week=("MIN_float", "sum"),
              reb_sum_week=("REB", "sum"),
              ast_sum_week=("AST", "sum"),
        )
        .sort_values(["PLAYER_ID", "season_dir", "team_week"])
    )

    gp_stats = weekly_player.groupby(["player_dir", "season_dir", "PLAYER_ID"])
    weekly_player["games_expanding_count"] = gp_stats["games_played"].cumsum()

    cols_to_delete = ["games_expanding_count"]
    for _col in ['pts', 'min', 'reb', 'ast']:
        weekly_player[f"{_col}_expanding_sum"] = gp_stats[f"{_col}_sum_week"].cumsum()
        weekly_player[f"{_col}_mean_up_to_week"] = (
            weekly_player[f"{_col}_expanding_sum"] / weekly_player["games_expanding_count"]
        )
        cols_to_delete.extend([f"{_col}_sum_week", f"{_col}_expanding_sum"])

    # Drop temporary helper columns
    weekly_player = weekly_player.drop(columns=cols_to_delete)

    # compute simple weekly shares (safe division)
    weekly_player["share_games_played"] = weekly_player["games_played"] / weekly_player["team_games_this_week"]
    weekly_player["share_games_started"] = weekly_player["games_started"] / weekly_player["games_played"]

    # --- cumulative (rolling across weeks) means per player ---
    # Sort by player + week so cumulative is correct
    weekly_player = weekly_player.sort_values(["player_dir", "season_dir", "PLAYER_ID", "team_week"]).reset_index(drop=True)

    return weekly_player

df_roll = build_player_week_df(df_all=df_bs)
print(df_roll.shape)
print(df_roll.PLAYER_NAME.unique())
df_roll.head()

(463, 19)
['Channing Frye' 'Stanley Johnson' 'Marquese Chriss' 'Frank Ntilikina'
 'Collin Sexton' 'Jaxson Hayes' 'Franz Wagner' 'Dyson Daniels'
 'Jarace Walker' 'Rob Dillingham' 'Egor Dëmin' 'Rudy Gay' 'Brandan Wright'
 'Joe Alexander' 'Jordan Hill' 'Al-Farouq Aminu' 'Brandon Knight'
 'Terrence Ross' 'Kentavious Caldwell-Pope' 'Jamal Crawford'
 'Nik Stauskas' 'DeSagana Diop' 'Chris Wilcox' 'T.J. Ford' 'Rafael Araujo']


,player_dir,season_dir,PLAYER_ID,PLAYER_NAME,TEAM_ID,team_week,games_played,games_started,min_mean_week,pts_mean_week,reb_mean_week,ast_mean_week,team_games_this_week,pts_mean_up_to_week,min_mean_up_to_week,reb_mean_up_to_week,ast_mean_up_to_week,share_games_played,share_games_started
0,101112,2005-06,101112,Channing Frye,1610612752,0,2,2,20.633333,9.50,7.50,1.50,3,9.500000,20.633333,7.500000,1.500000,0.666667,1.00
1,101112,2005-06,101112,Channing Frye,1610612752,1,4,3,16.254167,9.25,4.75,0.00,4,9.333333,17.713889,5.666667,0.500000,1.000000,0.75
2,101112,2005-06,101112,Channing Frye,1610612752,2,4,4,29.870833,18.75,5.75,1.75,4,13.100000,22.576667,5.700000,1.000000,1.000000,1.00
3,101112,2005-06,101112,Channing Frye,1610612752,3,4,4,35.533333,18.75,6.25,1.25,4,14.714286,26.278571,5.857143,1.071429,1.000000,1.00
4,101112,2005-06,101112,Channing Frye,1610612752,4,4,4,29.883333,14.25,6.25,0.75,4,14.611111,27.079630,5.944444,1.000000,1.000000,1.00


In [9]:
(
    df_roll
    .groupby(['season_dir', 'PLAYER_NAME'])
    [['pts_mean_up_to_week', 'reb_mean_up_to_week', 'ast_mean_up_to_week']]
    .last()
    .round(1)
    .reset_index().sort_values('season_dir')
)

,season_dir,PLAYER_NAME,pts_mean_up_to_week,reb_mean_up_to_week,ast_mean_up_to_week
0,2000-01,Jamal Crawford,4.6,1.5,2.3
1,2001-02,DeSagana Diop,1.4,0.9,0.3
2,2002-03,Chris Wilcox,3.7,2.3,0.5
3,2003-04,T.J. Ford,7.1,3.2,6.5
4,2004-05,Rafael Araujo,3.3,3.1,0.3
5,2005-06,Channing Frye,12.3,5.8,0.8
6,2006-07,Rudy Gay,10.8,4.5,1.3
7,2007-08,Brandan Wright,4.0,2.6,0.2
8,2008-09,Joe Alexander,4.7,1.9,0.7
9,2009-10,Jordan Hill,5.2,3.7,0.4


In [10]:
import pandas as pd
import json
import glob
import os

def get_first_seasons(directory='data/career'):
    all_first_seasons = []

    # Path to all json files in the directory
    file_paths = glob.glob(os.path.join(directory, "*.json"))

    for path in file_paths:
        with open(path, 'r') as f:
            data = json.load(f)

            # Usually NBA career data is in resultSets[0] or similar
            # If the format matches standard NBA API 'SeasonTotalsRegularSeason'
            if 'resultSets' in data:
                # Find the correct result set for season totals
                for rs in data['resultSets']:
                    if rs['name'] == 'SeasonTotalsRegularSeason':
                        df_career = pd.DataFrame(rs['rowSet'], columns=rs['headers'])
                        if not df_career.empty:
                            # Sort by SEASON_ID to ensure chronological order
                            df_career = df_career.sort_values('SEASON_ID')
                            # Take the first row (first season)
                            all_first_seasons.append(df_career.iloc[0])
                        break
            # Handle other possible formats if necessary

    if not all_first_seasons:
        return pd.DataFrame()

    return pd.DataFrame(all_first_seasons).reset_index(drop=True).sort_values('SEASON_ID')

# Usage
df_first_seasons = get_first_seasons()
df_first_seasons[["SEASON_ID", "TEAM_ABBREVIATION", "GP", "GS", "MIN", "PTS", "AST", "REB"]]

,SEASON_ID,TEAM_ABBREVIATION,GP,GS,MIN,PTS,AST,REB
13,2000-01,CHI,61,8,17.2,4.6,2.3,1.5
23,2001-02,CLE,18,1,6.1,1.4,0.3,0.9
5,2002-03,LAC,46,3,10.4,3.7,0.5,2.3
6,2003-04,MIL,55,55,26.8,7.1,6.5,3.2
1,2004-05,TOR,59,41,12.5,3.3,0.3,3.1
18,2005-06,NYK,65,14,24.2,12.3,0.8,5.8
7,2006-07,MEM,78,43,27.0,10.8,1.3,4.5
9,2007-08,GSW,38,6,9.9,4.0,0.2,2.6
0,2008-09,MIL,59,0,12.1,4.7,0.7,1.9
2,2009-10,NYK,24,0,10.5,4.0,0.3,2.5


# anthro

In [11]:
from datetime import datetime
import pandas as pd

def age_at_draft(birthdate_str, draft_year):
    birth = datetime.fromisoformat(birthdate_str)
    draft_day = datetime(draft_year, 6, 1)

    years = draft_day.year - birth.year
    if (draft_day.month, draft_day.day) < (birth.month, birth.day):
        years -= 1
    return years


def height_cm(height_wo_shoes, height_str):
    if height_wo_shoes and not pd.isna(height_wo_shoes):
        inches = height_wo_shoes
    else:
        feet, inch = height_str.split('-')
        inches = int(feet) * 12 + int(inch)

    return inches * 2.54


def weight_kg(weight_y, weight_x):
    pounds = weight_y if (weight_y and not pd.isna(weight_y)) else weight_x
    return pounds * 0.45359237


# --- Apply to df_cpi ---

df_cpi["AGE_AT_DRAFT"] = df_cpi.apply(
    lambda r: age_at_draft(r["BIRTHDATE"], r["DRAFT_YEAR"]),
    axis=1
)

df_cpi["HEIGHT_CM"] = df_cpi.apply(
    lambda r: height_cm(r["HEIGHT_WO_SHOES"], r["HEIGHT"]),
    axis=1
)

df_cpi["WEIGHT_KG"] = df_cpi.apply(
    lambda r: weight_kg(r["WEIGHT_y"], r["WEIGHT_x"]),
    axis=1
)
df_cpi["position_ru"] = df_cpi.POSITION.map({'Guard': "защитник", 'Center': "центр", 'Forward-Center': "центр", 'Forward-Guard': "винг", 'Forward': "винг",
       'Guard-Forward' :"защитник", 'Center-Forward': "центр"})

In [12]:
df_cpi.groupby('POSITION').agg({"LAST_NAME": list})

,LAST_NAME
POSITION,
Center,"[Diop, Araujo]"
Center-Forward,[Hayes]
Forward,"[Alexander, Aminu, Chriss, Wagner, Walker]"
Forward-Center,"[Wilcox, Frye, Wright, Hill]"
Forward-Guard,"[Gay, Johnson]"
Guard,"[Crawford, Ford, Knight, Caldwell-Pope, Stausk..."
Guard-Forward,[Ross]


In [13]:
df_cpi["last_name_asterisk"] = df_cpi.apply(lambda r: r.LAST_NAME + ("*" if pd.isna(r.PLAYER_ID) else ""), axis=1)
print(list(df_cpi.last_name_asterisk))

['Crawford', 'Diop', 'Wilcox', 'Ford', 'Araujo*', 'Frye', 'Gay', 'Wright', 'Alexander', 'Hill', 'Aminu', 'Knight', 'Ross', 'Caldwell-Pope', 'Stauskas', 'Johnson', 'Chriss', 'Ntilikina*', 'Sexton', 'Hayes', 'Wagner*', 'Daniels', 'Walker', 'Dillingham', 'Dëmin']


In [14]:
import altair as alt
import pandas as pd


# define domain around the mean
h_min, h_max = df_cpi["HEIGHT_CM"].min() * 0.99, df_cpi["HEIGHT_CM"].max() * 1
w_min, w_max = df_cpi["WEIGHT_KG"].min() * 0.99, df_cpi["WEIGHT_KG"].max() * 1

base = alt.Chart(df_cpi).encode(
    x=alt.X('HEIGHT_CM:Q', 
            scale=alt.Scale(domain=[h_min, h_max]),
            title='Рост (см)'),
    y=alt.Y('WEIGHT_KG:Q', 
            scale=alt.Scale(domain=[w_min, w_max]),
            title='Вес (кг)'),
    color=alt.Color('position_ru:N', title='Амплуа'),
    size=alt.Size(
        'AGE_AT_DRAFT:Q',
        scale=alt.Scale(domain=[18, 23], range=[50, 300]),
        title='Полных лет на момент драфта'
    ),
    tooltip=[
        alt.Tooltip('LAST_NAME:N', title='Name'),
        alt.Tooltip('POSITION:N', title='Position'),
        alt.Tooltip('AGE_AT_DRAFT:Q', title='Age at draft'),
        alt.Tooltip('HEIGHT_CM:Q', title='Height (cm)'),
        alt.Tooltip('WEIGHT_KG:Q', title='Weight (kg)')
    ]
)

points = base.mark_circle(opacity=0.6)

labels = alt.Chart(df_cpi).mark_text(
    align='left',
    baseline='middle',
    dx=5,  # horizontal offset
    dy=0,   # vertical offset (if you want adjust you can change)
    fontSize=18,     # ← set font size (in pixels)
    angle=330,
).encode(
    x='HEIGHT_CM:Q',
    y='WEIGHT_KG:Q',
    text='last_name_asterisk:N'
)

chart = points + labels

chart = chart.properties(
    width=1400,
    height=700,
    title='Антропометрия'
).interactive()

chart

alt.LayerChart(...)

# 1st season

In [15]:
df_roll.PLAYER_NAME.unique(), df_roll.season_dir.min()

(array(['Channing Frye', 'Stanley Johnson', 'Marquese Chriss',
        'Frank Ntilikina', 'Collin Sexton', 'Jaxson Hayes', 'Franz Wagner',
        'Dyson Daniels', 'Jarace Walker', 'Rob Dillingham', 'Egor Dëmin',
        'Rudy Gay', 'Brandan Wright', 'Joe Alexander', 'Jordan Hill',
        'Al-Farouq Aminu', 'Brandon Knight', 'Terrence Ross',
        'Kentavious Caldwell-Pope', 'Jamal Crawford', 'Nik Stauskas',
        'DeSagana Diop', 'Chris Wilcox', 'T.J. Ford', 'Rafael Araujo'],
       dtype=object),
 '2000-01')

In [16]:
import altair as alt
import pandas as pd

def plot_1st_season(df_roll: pd.DataFrame, col_to_plot: str, player_ids_to_highlight: list, col_to_plot_ru: str=None):
    """
    df_roll: the rolling/week dataframe you built
    col_to_plot: name of the column to plot in boxplot (e.g. "pts_mean_week", "min_mean_rolling", etc.)
    player_ids_to_highlight: list of PLAYER_IDs (ints) whose per-week values will be overlaid as line + points
    """
    base = df_roll.copy()
    # ensure week is treated as ordinal (or integer)
    base["week_str"] = base["team_week"].astype(str)

    # --- Box plot of distribution across all players by week ---
    box = alt.Chart(base).mark_boxplot(
        extent='min-max',
        color='lightgrey',
        opacity=0.5
    ).encode(
        x=alt.X("team_week:O", title="Блок из 4х игр"),
        y=alt.Y(f"{col_to_plot}:Q", title=col_to_plot_ru if col_to_plot_ru else col_to_plot),
    )

    # --- For highlighted players: extract their per-week values ---
    highlight_df = base[base["PLAYER_ID"].isin(player_ids_to_highlight)]

    line = alt.Chart(highlight_df).mark_line().encode(
        x=alt.X("team_week:O"),
        y=alt.Y(f"{col_to_plot}:Q"),
        color=alt.Color("PLAYER_NAME:O", scale=alt.Scale(scheme="set1"), title="Игрок"),
    )

    points = alt.Chart(highlight_df).mark_point(filled=True, size=60).encode(
        x=alt.X("team_week:O"),
        y=alt.Y(f"{col_to_plot}:Q"),
        size=alt.Size("share_games_started:Q", 
                        title="Доля выходов в старте"),
        color=alt.Color("share_games_played:Q",
                        title="Доля сыгранных игр"),
#         shape=alt.Shape("PLAYER_ID:N", legend=None)
    )

    chart = box + points + line
    chart = chart.properties(width=1200, height=700)
    return chart

plot_1st_season(df_roll, col_to_plot='min_mean_week', player_ids_to_highlight=[101112, 1642265, 1642856,1630532], col_to_plot_ru="Среднее кол-во минут в матчах блока")

alt.LayerChart(...)

In [17]:
plot_1st_season(df_roll, col_to_plot='min_mean_up_to_week', player_ids_to_highlight=[101112, 1642265, 1642856,1630532], col_to_plot_ru="Среднее кол-во минут в матчах от начала сезона до этого блока")

alt.LayerChart(...)

In [18]:
plot_1st_season(df_roll, col_to_plot='pts_mean_week', player_ids_to_highlight=[101112, 1642265, 1642856,1630532], col_to_plot_ru="Среднее кол-во очков в матчах блока")

alt.LayerChart(...)

In [19]:
df_roll.query("(team_week == 17) or (team_week==16)").sort_values('pts_mean_week', ascending=False).head(5)

,player_dir,season_dir,PLAYER_ID,PLAYER_NAME,TEAM_ID,team_week,games_played,games_started,min_mean_week,pts_mean_week,reb_mean_week,ast_mean_week,team_games_this_week,pts_mean_up_to_week,min_mean_up_to_week,reb_mean_up_to_week,ast_mean_up_to_week,share_games_played,share_games_started
96,1629012,2018-19,1629012,Collin Sexton,1610612739,17,4,4,36.362500,26.0,2.50,2.25,4,16.028169,31.031925,2.971831,2.887324,1.0,1.0
95,1629012,2018-19,1629012,Collin Sexton,1610612739,16,4,4,32.758333,21.0,1.75,4.50,4,15.432836,30.713682,3.000000,2.925373,1.0,1.0
54,1627737,2016-17,1627737,Marquese Chriss,1610612756,17,4,4,30.629167,17.5,7.50,0.50,4,8.676056,20.412911,3.901408,0.647887,1.0,1.0
218,200752,2006-07,200752,Rudy Gay,1610612763,16,4,4,36.495833,17.5,6.25,2.00,4,10.731343,26.252488,4.388060,1.164179,1.0,1.0
135,1630532,2021-22,1630532,Franz Wagner,1610612753,17,4,4,31.470833,14.0,4.75,2.25,4,15.408451,31.607042,4.633803,2.929577,1.0,1.0


In [20]:
plot_1st_season(df_roll, col_to_plot='pts_mean_up_to_week', player_ids_to_highlight=[101112, 1642265, 1642856,1630532], col_to_plot_ru="Среднее кол-во очков в матчах от начала сезона до этого блока")

alt.LayerChart(...)

# career

In [27]:
import os
def load_career_data(directory='data/career'):
    all_seasons = []
    file_paths = glob.glob(os.path.join(directory, "*.json"))

    for path in file_paths:
        with open(path, 'r') as f:
            data = json.load(f)

            if 'resultSets' in data:
                for rs in data['resultSets']:
                    if rs['name'] == 'SeasonTotalsRegularSeason':
                        df_p = pd.DataFrame(rs['rowSet'], columns=rs['headers'])
                        # Identify player by name from the first available record if not in headers
                        all_seasons.append(df_p)
                        break

    if not all_seasons:
        return pd.DataFrame()

    df_career = pd.concat(all_seasons, ignore_index=True)

    # Calculate years of experience (0 = rookie season)
    df_career = df_career.sort_values(['PLAYER_ID', 'SEASON_ID'])
    df_career['experience'] = df_career.groupby('PLAYER_ID').cumcount()

    # Map player names if they are missing in the career files but present in our other data
    # (Assuming df_cpi or similar is available in your namespace)
    name_map = df_cpi.set_index('player_id')['DISPLAY_FIRST_LAST'].to_dict()
    df_career['PLAYER_NAME'] = df_career['PLAYER_ID'].map(name_map)

    return df_career


df_career = load_career_data()
print(df_career.shape)
players_with_5_years = df_career.groupby('PLAYER_NAME').filter(lambda x: len(x) >= 5)['PLAYER_NAME'].unique()
print(f"players with 5+ years: {len(players_with_5_years)} - {players_with_5_years}")

# Filter for these players and take only their first 5 years (experience 0 to 4)
df_5years = df_career[
    (df_career['PLAYER_NAME'].isin(players_with_5_years)) &
    (df_career['experience'] < 5)
]
print(df_5years.shape)
df_5years#.head()

(256, 29)
players with 5+ years: 19 - ['Jamal Crawford' 'DeSagana Diop' 'Chris Wilcox' 'T.J. Ford'
 'Channing Frye' 'Rudy Gay' 'Brandan Wright' 'Jordan Hill'
 'Al-Farouq Aminu' 'Brandon Knight' 'Terrence Ross'
 'Kentavious Caldwell-Pope' 'Nik Stauskas' 'Stanley Johnson'
 'Marquese Chriss' 'Frank Ntilikina' 'Collin Sexton' 'Jaxson Hayes'
 'Franz Wagner']
(95, 29)


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,experience,PLAYER_NAME
135,2037,2000-01,00,1610612741,CHI,21.0,61,8,17.2,1.8,5.0,0.352,0.7,1.9,0.350,0.4,0.6,0.794,0.1,1.3,1.5,2.3,0.7,0.2,1.4,1.1,4.6,0,Jamal Crawford
136,2037,2001-02,00,1610612741,CHI,22.0,23,6,20.9,3.9,8.1,0.476,1.1,2.5,0.448,0.4,0.6,0.769,0.2,1.3,1.5,2.4,0.8,0.2,1.4,0.8,9.3,1,Jamal Crawford
137,2037,2002-03,00,1610612741,CHI,23.0,80,31,24.9,4.2,10.1,0.413,1.1,3.0,0.355,1.3,1.6,0.806,0.3,2.1,2.3,4.2,1.0,0.3,1.7,1.6,10.7,2,Jamal Crawford
138,2037,2003-04,00,1610612741,CHI,24.0,80,73,35.1,6.4,16.5,0.386,2.1,6.5,0.317,2.5,3.0,0.833,0.6,3.0,3.5,5.1,1.4,0.4,2.4,2.0,17.3,3,Jamal Crawford
139,2037,2004-05,00,1610612752,NYK,25.0,70,67,38.4,6.2,15.7,0.398,2.6,7.3,0.361,2.6,3.1,0.843,0.5,2.4,2.9,4.3,1.3,0.3,2.1,1.9,17.7,4,Jamal Crawford
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,1630532,2021-22,00,1610612753,ORL,20.0,79,79,30.7,5.8,12.3,0.468,1.2,3.4,0.354,2.4,2.8,0.863,1.1,3.4,4.5,2.9,0.9,0.4,1.5,2.1,15.2,0,Franz Wagner
227,1630532,2022-23,00,1610612753,ORL,21.0,80,80,32.6,6.8,14.0,0.485,1.6,4.5,0.361,3.4,4.0,0.842,0.9,3.2,4.1,3.5,1.0,0.2,2.1,2.3,18.6,1,Franz Wagner
228,1630532,2023-24,00,1610612753,ORL,22.0,72,72,32.5,7.3,15.2,0.482,1.3,4.6,0.281,3.8,4.4,0.850,1.0,4.3,5.3,3.7,1.1,0.4,1.9,2.3,19.7,2,Franz Wagner
229,1630532,2024-25,00,1610612753,ORL,23.0,60,60,33.7,9.0,19.4,0.463,1.7,5.9,0.295,4.5,5.2,0.871,0.9,4.8,5.7,4.7,1.3,0.4,2.3,2.5,24.2,3,Franz Wagner


In [30]:
def plot_career_stats(df_career: pd.DataFrame, col_to_plot: str, player_ids_to_highlight: list, col_to_plot_ru: str=None):
    """
    Plots career trajectory distribution with highlighted players.
    col_to_plot: 'MIN', 'PTS', 'REB', 'AST', etc.
    """
    base_df = df_career.copy()

    # ---- add starter share (GS/GP) for point size ----
    base_df["start_pct"] = (base_df["GS"] / base_df["GP"]).replace([np.inf, -np.inf], np.nan).fillna(0)
    base_df["start_pct"] = base_df["start_pct"].clip(0, 1)
    base_df["year_ru"] = base_df.experience + 1

    y_min = base_df[col_to_plot].min()
    y_max = base_df[col_to_plot].max()

    pad = 0.05 * (y_max - y_min)  # 5% padding (use 0.02, 0.1, etc.)
    y_domain = [y_min - pad, y_max + pad]

    # Distribution of all 8th picks by year of experience
    box = alt.Chart(base_df).mark_boxplot(
        extent='min-max',
        color='lightgrey',
        opacity=0.5
    ).encode(
        x=alt.X("year_ru:O", title="Сезон игрока в НБА"),
        y=alt.Y(f"{col_to_plot}:Q",
                scale=alt.Scale(domainMin=y_min - pad, domainMax=y_max + pad),
                title=col_to_plot_ru if col_to_plot_ru else col_to_plot),
    )

    # Highlight specific players
    highlight_df = base_df[base_df["PLAYER_ID"].isin(player_ids_to_highlight)]

    line = alt.Chart(highlight_df).mark_line().encode(
        x=alt.X("year_ru:O"),
        y=alt.Y(f"{col_to_plot}:Q"),
        color=alt.Color("PLAYER_NAME:N", scale=alt.Scale(scheme="set1"), title="Игрок"),
    )

    points = alt.Chart(highlight_df).mark_point(filled=True, size=60).encode(
        x=alt.X("year_ru:O"),
        y=alt.Y(f"{col_to_plot}:Q", ),
        color=alt.Color("PLAYER_NAME:N"),
        size=alt.Size(
            "start_pct:Q",
            title="Доля выходов в старте",
            scale=alt.Scale(domain=[0, 1], range=[30, 300])
        ),
        tooltip=[
            alt.Tooltip("PLAYER_NAME:N"),
            alt.Tooltip("SEASON_ID:N"),
            alt.Tooltip("TEAM_ABBREVIATION:N"),
            alt.Tooltip("GP:Q"),
            alt.Tooltip("GS:Q"),
            alt.Tooltip("start_pct:Q", title="GS/GP", format=".0%"),
            alt.Tooltip(f"{col_to_plot}:Q"),
        ],
    )

    chart = (line + points + box).properties(
        width=1200,
        height=700,
        title=f"Первые 5 лет карьеры"
    ).interactive()

    return chart


# Example usage:
# plot_career_stats(df_career, col_to_plot='PTS', player_ids_to_highlight=[2037, 200752, 101112])
plot_career_stats(df_5years, col_to_plot='MIN', player_ids_to_highlight=[2037, 200752, 101112, 203484, 1630532], col_to_plot_ru="Среднее количество минут за сезон")

alt.LayerChart(...)

In [31]:
plot_career_stats(df_5years, col_to_plot='PTS', player_ids_to_highlight=[2037, 200752, 101112, 203484, 1630532], col_to_plot_ru="Среднее количество очков за сезон")

alt.LayerChart(...)

In [33]:
df_career.query('experience == 0').sort_values('STL', ascending=False)

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,experience,PLAYER_NAME
51,2551,2003-04,00,1610612749,MIL,21.0,55,55,26.8,2.8,7.2,0.384,0.1,0.4,0.238,1.5,1.8,0.816,0.7,2.5,3.2,6.5,1.1,0.1,2.5,2.2,7.1,0,T.J. Ford
255,1642856,2025-26,00,1610612751,BKN,19.0,35,28,24.7,3.5,8.8,0.404,2.4,6.1,0.395,0.9,1.1,0.838,0.5,2.6,3.1,3.5,0.9,0.3,1.8,1.8,10.4,0,Egor Dëmin
226,1630532,2021-22,00,1610612753,ORL,20.0,79,79,30.7,5.8,12.3,0.468,1.2,3.4,0.354,2.4,2.8,0.863,1.1,3.4,4.5,2.9,0.9,0.4,1.5,2.1,15.2,0,Franz Wagner
59,200752,2006-07,00,1610612763,MEM,20.0,78,43,27.0,4.1,9.7,0.422,0.7,1.8,0.364,1.9,2.7,0.727,1.2,3.3,4.5,1.3,0.9,0.9,1.8,2.5,10.8,0,Rudy Gay
157,203484,2013-14,00,1610612765,DET,21.0,80,41,19.8,2.3,5.8,0.396,0.7,2.3,0.319,0.6,0.8,0.770,0.5,1.5,2.0,0.7,0.9,0.2,0.4,1.8,5.9,0,Kentavious Caldwell-Pope
80,1628373,2017-18,00,1610612752,NYK,19.0,78,9,21.9,2.3,6.4,0.364,0.6,2.0,0.318,0.6,0.9,0.721,0.5,1.8,2.3,3.2,0.8,0.2,1.7,2.3,5.9,0,Frank Ntilikina
231,1627737,2016-17,00,1610612756,PHX,19.0,82,75,21.3,3.5,7.7,0.449,0.9,2.7,0.321,1.4,2.2,0.624,1.2,3.1,4.2,0.7,0.8,0.9,1.3,3.2,9.2,0,Marquese Chriss
190,1626169,2015-16,00,1610612765,DET,20.0,73,6,23.1,3.1,8.2,0.375,0.9,2.8,0.307,1.1,1.4,0.784,0.8,3.4,4.2,1.6,0.8,0.2,1.6,2.4,8.1,0,Stanley Johnson
17,202688,2011-12,00,1610612765,DET,20.0,66,60,32.3,4.8,11.7,0.415,1.6,4.2,0.380,1.6,2.1,0.759,0.5,2.7,3.2,3.8,0.7,0.2,2.6,2.3,12.8,0,Brandon Knight
32,1630700,2022-23,00,1610612740,NOP,20.0,59,11,17.7,1.5,3.5,0.418,0.5,1.5,0.314,0.4,0.7,0.650,0.6,2.6,3.2,2.3,0.7,0.2,1.0,1.7,3.8,0,Dyson Daniels


In [39]:
df_career.query('(experience == 0) and (FG3M > 2)').sort_values('STL', ascending=False)

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,experience,PLAYER_NAME
255,1642856,2025-26,00,1610612751,BKN,19.0,35,28,24.7,3.5,8.8,0.404,2.4,6.1,0.395,0.9,1.1,0.838,0.5,2.6,3.1,3.5,0.9,0.3,1.8,1.8,10.4,0,Egor Dëmin
